In [ ]:
# | default_exp schedulers/noise

# Imports

In [ ]:
# | export


import torch
from einops import rearrange
from torch import nn

# Schedulers

In [ ]:
# | export


# TODO: Clean and make more usable
class GaussianNoiseScheduler(nn.Module):
    def __init__(self, betas):
        super().__init__()

        self.betas = betas
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - self.alphas_cumprod)

    def add_noise(self, x0_batch, t):
        standard_noise = torch.randn(x0_batch.shape, device=x0_batch.device)

        sqrt_alphas_cumprod = self.sqrt_alphas_cumprod.to(x0_batch.device)
        sqrt_one_minus_alphas_cumprod = self.sqrt_one_minus_alphas_cumprod.to(x0_batch.device)

        momentum_term = rearrange(sqrt_alphas_cumprod[t], "n -> n 1 1 1") * x0_batch
        noise_term = rearrange(sqrt_one_minus_alphas_cumprod[t], "n -> n 1 1 1") * standard_noise

        return momentum_term + noise_term, standard_noise

    def remove_noise(self, xt_batch, pred_batch, t):  # pred_batch is the noise prediction, not the denoised image
        x0_batch = (xt_batch - (self.sqrt_one_minus_alphas_cumprod[t] * pred_batch)) / self.sqrt_alphas_cumprod[t]
        x0_batch = x0_batch.clamp(-1.0, 1.0)

        mean_batch = xt_batch - ((self.betas[t] * pred_batch) / (self.sqrt_one_minus_alphas_cumprod[t])) / torch.sqrt(
            self.alphas[t]
        )

        variance_batch = self.betas[t] * (1.0 - self.alphas_cumprod[t - 1]) / (1.0 - self.alphas_cumprod[t])
        sigma_batch = torch.sqrt(variance_batch)
        standard_noise = torch.randn(xt_batch.shape, device=xt_batch.device)

        # Return only mean when t = 0
        sigma_batch[t == 0] = 0

        return x0_batch, mean_batch + sigma_batch * standard_noise

In [ ]:
T = 200
betas = torch.linspace(0.0001, 0.02, T + 1)
scheduler = GaussianNoiseScheduler(betas)

x0 = torch.zeros(1, 3, 32, 32)
t = torch.tensor([10])
xt, noise = scheduler.add_noise(x0, t)

x0_hat, xt_minus_1 = scheduler.remove_noise(xt, noise, t)
assert torch.allclose(x0, x0_hat)

# nbdev

In [ ]:
!nbdev_export